## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import rand

# File location and type
file_location = "/FileStore/tables/ml_q2.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

settlement = df.filter(df['has_settlement'] == 1)
no_settlement = df.filter(df['has_settlement'] == 0)

no_settlement = no_settlement.sample(False, 0.3)
data = settlement.union(no_settlement)


indexer = StringIndexer(inputCol="complaint_category", outputCol="categoryIndex")
indexed = indexer.fit(data).transform(data)

feature_cols = ['allegation_count', 'categoryIndex']
vectorAssembler = VectorAssembler(inputCols = ['allegation_count', 'categoryIndex'], outputCol = 'features')
v_df = vectorAssembler.transform(indexed)
v_df = v_df.select(['features', 'has_settlement'])
#v_df.show()


(trainingData, testData) = v_df.randomSplit([0.7, 0.3])

dt = DecisionTreeClassifier(labelCol="has_settlement", featuresCol="features")
model = dt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol="has_settlement", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g " % accuracy)

tree_model = model.toDebugString
for i, feat in enumerate(feature_cols):
    tree_model = tree_model.replace('feature ' + str(i), feat)
print(tree_model)




+-----+--------------+----------------+--------------------+-------------+
 id|has_settlement|allegation_count| complaint_category|categoryIndex|
+-----+--------------+----------------+--------------------+-------------+
11190| 1| 82| Use Of Force| 1.0|
15194| 1| 27|Operation/Personn...| 0.0|
15663| 1| 9| Illegal Search| 2.0|
15663| 1| 9| Illegal Search| 2.0|
23506| 1| 7| Use Of Force| 1.0|
25084| 1| 1|Operation/Personn...| 0.0|
29018| 1| 49| Illegal Search| 2.0|
 541| 1| 11| Use Of Force| 1.0|
21856| 1| 35|Operation/Personn...| 0.0|
23870| 1| 18| Use Of Force| 1.0|
31503| 1| 4|Conduct Unbecomin...| 4.0|
 2570| 1| 9| Illegal Search| 2.0|
 9712| 1| 31|Operation/Personn...| 0.0|
10014| 1| 40| Illegal Search| 2.0|
28138| 1| 42| Illegal Search| 2.0|
32293| 1| 24|Operation/Personn...| 0.0|
 367| 1| 3| Illegal Search| 2.0|
 5572| 1| 13| Illegal Search| 2.0|
19639| 1| 37| Use Of Force| 1.0|
29063| 1| 8|Operation/Personn...| 0.0|
+-----+--------------+----------------+--------------------+-------------+
only showing top 20 rows

Accuracy = 0.817406 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4b26bac852ee4ea24be0) of depth 5 with 53 nodes
 If (allegation_count <= 14.5)
 If (allegation_count <= 5.5)
 If (allegation_count <= 3.5)
 If (categoryIndex in {4.0,6.0,7.0,8.0,10.0,12.0})
 If (categoryIndex in {4.0,7.0,8.0,10.0,12.0})
 Predict: 0.0
 Else (categoryIndex not in {4.0,7.0,8.0,10.0,12.0})
 Predict: 0.0
 Else (categoryIndex not in {4.0,6.0,7.0,8.0,10.0,12.0})
 If (categoryIndex in {1.0,2.0,3.0,5.0,9.0,11.0,13.0})
 Predict: 0.0
 Else (categoryIndex not in {1.0,2.0,3.0,5.0,9.0,11.0,13.0})
 Predict: 0.0
 Else (allegation_count > 3.5)
 If (categoryIndex in {4.0,8.0,10.0,11.0,14.0})
 Predict: 0.0
 Else (categoryIndex not in {4.0,8.0,10.0,11.0,14.0})
 If (categoryIndex in {1.0,2.0,3.0,5.0,6.0,7.0,9.0,12.0})
 Predict: 0.0
 Else (categoryIndex not in {1.0,2.0,3.0,5.0,6.0,7.0,9.0,12.0})
 Predict: 0.0
 Else (allegation_count > 5.5)
 If (categoryIndex in {1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0})
 If (categoryIndex in {3.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0})
 If (categoryIndex in {4.0,6.0,8.0,10.0,11.0,12.0})
 Predict: 0.0
 Else (categoryIndex not in {4.0,6.0,8.0,10.0,11.0,12.0})
 Predict: 0.0
 Else (categoryIndex not in {3.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0})
 If (allegation_count <= 8.5)
 Predict: 0.0
 Else (allegation_count > 8.5)
 Predict: 0.0
 Else (categoryIndex not in {1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0})
 If (allegation_count <= 9.5)
 If (allegation_count <= 7.5)
 Predict: 0.0
 Else (allegation_count > 7.5)
 Predict: 0.0
 Else (allegation_count > 9.5)
 If (allegation_count <= 11.5)
 Predict: 0.0
 Else (allegation_count > 11.5)
 Predict: 0.0
 Else (allegation_count > 14.5)
 If (categoryIndex in {0.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0})
 If (allegation_count <= 28.5)
 If (categoryIndex in {6.0,10.0})
 Predict: 0.0
 Else (categoryIndex not in {6.0,10.0})
 If (categoryIndex in {3.0,4.0,5.0,7.0,8.0})
 Predict: 0.0
 Else (categoryIndex not in {3.0,4.0,5.0,7.0,8.0})
 Predict: 0.0
 Else (allegation_count > 28.5)
 If (categoryIndex in {4.0,8.0,10.0})
 Predict: 0.0
 Else (categoryIndex not in {4.0,8.0,10.0})
 If (allegation_count <= 46.5)
 Predict: 0.0
 Else (allegation_count > 46.5)
 Predict: 1.0
 Else (categoryIndex not in {0.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0})
 If (allegation_count <= 46.5)
 If (categoryIndex in {2.0})
 If (allegation_count <= 29.5)
 Predict: 0.0
 Else (allegation_count > 29.5)
 Predict: 1.0
 Else (categoryIndex not in {2.0})
 If (allegation_count <= 35.5)
 Predict: 0.0
 Else (allegation_count > 35.5)
 Predict: 1.0
 Else (allegation_count > 46.5)
 If (categoryIndex in {1.0})
 Predict: 0.0
 Else (categoryIndex not in {1.0})
 Predict: 1.0

In [3]:
display(indexed.filter(indexed['categoryIndex'] == 13))

id,has_settlement,allegation_count,complaint_category,categoryIndex
14593,1,3,First Amendment,13.0
11670,0,1,First Amendment,13.0
25775,0,1,First Amendment,13.0
28044,0,2,First Amendment,13.0


In [4]:
# Create a view or table

temp_table_name = "officer_allegations_new_csv"

df.createOrReplaceTempView(temp_table_name)

In [5]:
%sql

/* Query the created temp table in a SQL cell */

select * from `officer_allegations_new_csv`

In [6]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "officer_allegations_new_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)